In [ ]:
from pathlib import Path
import os
import json 
import pandas as pd
import numpy as np
import datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [ ]:
# setup gcp using terraform
# !terraform init

In [60]:
from mermaid import display_graph
display_graph()

In [ ]:
# generate random.csv
num_records = 1000
dim_values =['a','b','c','d']
columns = ['amt','msr']

assert num_records % len(dim_values) == 0

(
    pd.DataFrame(np.random.randint(0,100,size=(num_records, len(columns))),columns=columns)
    .assign(dim=int(num_records/len(dim_values)) * dim_values)
    .head()
    # .assign(asof=datetime.datetime.now())
    [['dim','amt','msr']]
    .to_csv('random.csv',index=False)
)


In [51]:
# save schema of random
data={'BigQuery Schema':[
    {"name": "dim","type": "STRING","mode": "NULLABLE"},
    {"name": "amt","type": "INTEGER","mode": "NULLABLE"},
    {"name": "msr","type": "INTEGER","mode": "NULLABLE"}
]}
with open('schema.json', 'w') as f:
    json.dump(data, f)

In [ ]:
# create network, buckets, big query dataset
# dataflow_job in main.tf commented out.  
!terraform apply -auto-approve

In [53]:
!gcloud storage cp random.csv schema.json gs://hca-hvu7470-202504041334-sbx-io

Copying file://random.csv to gs://hca-hvu7470-202504041334-sbx-io/random.csv
Copying file://schema.json to gs://hca-hvu7470-202504041334-sbx-io/schema.json
  
......................

Average throughput: 7.6kiB/s


In [ ]:
file = Path(os.environ['APPDATA']) / r"gcloud\application_default_credentials.json"
with open(file) as f:
    PROJECT=json.load(f)['quota_project_id']


from google.cloud import bigquery

def qry(sql):    
    return bigquery.Client(PROJECT).query(sql).to_dataframe()

qry('''select * from hca-hvu7470-202504041334-sbx.dataflow.random''')
qry('''select * from hca-hvu7470-202504041334-sbx.dataflow.random_err''')

,dim,amt,msr


,RawContent,ErrorMsg


In [ ]:
# dataflow_job in main.tf included  
!terraform apply -auto-approve

# todo - how to 
# https://cloud.google.com/sdk/gcloud/reference/dataflow/jobs/run
# https://cloud.google.com/sdk/gcloud/reference/dataflow/yaml/run

# templates 
# https://console.cloud.google.com/storage/browser/_details/dataflow-templates/latest/GCS_CSV_to_BigQuery

In [62]:
qry('''select count(*) from hca-hvu7470-202504041334-sbx.dataflow.random''')
qry('''select * from hca-hvu7470-202504041334-sbx.dataflow.random''')
qry('''select * from hca-hvu7470-202504041334-sbx.dataflow.random_err''')

,f0_
0,10


,dim,amt,msr
0,a,77,1
1,a,31,4
2,b,39,55
3,c,33,8
4,d,29,65
5,a,77,1
6,a,31,4
7,b,39,55
8,c,33,8
9,d,29,65


,RawContent,ErrorMsg
